In [2]:
library(ggplot2)
library(reshape2)
library(dplyr)
library(stringr)
library(tidyr)
theme_set(theme_bw())
options(repr.plot.width=7, repr.plot.height=4)


Attaching package: ‘tidyr’

The following object is masked from ‘package:reshape2’:

    smiths



In [8]:
identities = read.delim('identities.tsv', sep='\t', stringsAsFactors=FALSE)
identities$quality = as.logical(identities$quality)
identities$restrict = as.logical(identities$restrict)

# Introduction

Here, I measure base frequencies and tabulate identity elements by clade. Biologists in the past have named things as identity elements if 100% of tRNAs have this element. I'll relax those restrictions to 98% among the quality and species-restricted tRNAs.

Terminology:
- An **identity element** is a base or base class (G, purine, etc.) located at a specific position.
- **Consensus** identity elements for a clade means that all subclades contain this identity element.
- **Diverged** positions for a clade means that all subclades contain an IDE at that position, even if the IDE differs. Of course, the IDE itself could be "zoomed out", to a lower specificity, and then the position wouldn't be described as diverged anymore. For example, if archaea and bacteria have A41 but eukarya have G41, then position 41 is diverged if you consider A41 and G41 to be separate IDEs, but not diverged if you call it as R41.
- **Unique** identity elements for a subclade means that all sister subclades do not have an IDE at this position.
- We could potentially have a case where certain subclades have an IDE at a position while others do not. These will be known as **shared** identity elements among two or more unrelated subclades.

## Base combination table

Reproduced from euk-Leu

Symbol   |   Bases  | Rule  | Rank
:--------|----------|-------|------
R | G A | Purine | 2
Y | U C | Pyrimidine | 2
K | G U | Keto | 2
M | A C | Amino | 2
S | G C | Strong | 2
W | A U | Weak | 2
B | C G U | not A | 3
D | G A U | not C | 3
H | A C U | not G | 3
V | G C A | not U | 3
R:Y | G:C A:U | Purine-Pyrimidine | 2
Y:R | C:G U:A | Pyrimidine-Purine | 2
S:S | G:C C:G | Strong | 2
W:W | A:U U:A | Weak | 2
W:O | G:U U:G | Wobble | 2
B:V | G:C C:G U:A | not A:U | 3
V:B | G:C C:G A:U | not U:A | 3
D:H | A:U U:A G:C | not C:G | 3
H:D | A:U U:A C:G | not G:C | 3
W:C | A:U U:A G:C C:G | Watson-Crick | 4
G:N | G:C G:U C:G U:G | G | 5
U:N | U:G U:A A:U G:U | U | 5
C:O | A:U U:A G:C C:G G:U U:G | Canonical pairs | 6
Y:Y | U:C C:U U:U C:C | Pyrimidine-pyrimidine mismatch | 7
R:R | A:G G:A A:A G:G | Purine-purine mismatch | 7
M:M | A:A G:G C:C U:U A:G A:C C:A C:U G:A U:C | Mismatch | 8

# Consensus eukaryotic identity elements

I will start with the easiest case: IDEs that are almost perfectly conserved among all eukaryotes.

In [4]:
positions = colnames(identities)[which(str_detect(colnames(identities), 'X\\d+\\.'))]
positions = c(positions, 'X8', 'X9', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X20a', 'X21', 'X26', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X44', 'X45', 'X46', 'X47', 'X48', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X73')

In [98]:
euk_freqs = identities %>% select(match(positions, colnames(identities))) %>%
  gather(positions, bases) %>%
  group_by(positions, bases) %>%
  tally() %>%
  group_by(positions) %>%
  mutate(freq=n/sum(n)) %>%
  group_by(positions) %>%
  summarize(A = sum(freq[bases == "A"]),
            C = sum(freq[bases == "C"]),
            G = sum(freq[bases == "G"]),
            U = sum(freq[bases == "U"]),
            Purine = sum(freq[bases %in% c("A", "G")]),
            Pyrimidine = sum(freq[bases %in% c("C", "U")]),
            Weak = sum(freq[bases %in% c("A", "U")]),
            Strong = sum(freq[bases %in% c("G", "C")]),
            Amino = sum(freq[bases %in% c("A", "C")]),
            Keto = sum(freq[bases %in% c("G", "U")]),
            B = sum(freq[bases %in% c("C", "G", "U")]),
            D = sum(freq[bases %in% c("A", "G", "U")]),
            H = sum(freq[bases %in% c("A", "C", "U")]),
            V = sum(freq[bases %in% c("A", "C", "G")]),
            D = sum(freq[bases %in% c("A", "G", "U")]),
            GC = sum(freq[bases == "G:C"]),
            AU = sum(freq[bases == "A:U"]),
            UA = sum(freq[bases == "U:A"]),
            CG = sum(freq[bases == "C:G"]),
            PurinePyrimidine = sum(freq[bases %in% c("A:U", "U:A", "C:G", "G:C")]),
            StrongPair = sum(freq[bases %in% c("G:C", "C:G")]),
            WeakPair = sum(freq[bases %in% c("A:U", "U:A")]),
            Wobble = sum(freq[bases %in% c("G:U", "U:G")]),
            Paired = sum(freq[bases %in% c("A:U", "U:A", "C:G", "G:C", "G:U", "U:G")]),
            Mismatched = sum(freq[bases %in% c("A:A", "G:G", "C:C", "U:U", "A:G", "A:C", "C:A", "C:U", "G:A", "U:C")])
            )

In [99]:
euk_freqs %>%
  melt %>% 
  filter(value > 0.95) %>%
  group_by(positions) %>% # remove duplicates
  filter(row_number(value) == 1)

Using positions as id variables


,positions,variable,value
1,X14,A,0.9942488
2,X21,A,0.982819
3,X58,A,0.9744916
4,X18,G,0.9891507
5,X19,G,0.9878808
6,X33,U,0.9771222
7,X54,U,0.9722147
8,X55,U,0.9893412
9,X8,U,0.9739654
10,X15,Purine,0.997079
